# Chi-Square Analysis: Mushroom Classification Dataset
## Research Questions: 
1. Is there an association between cap color and toxicity?
2. Is there an association between odor and toxicity?

**Dataset**: Mushroom characteristics (UCI Machine Learning Repository)  
**Test**: Chi-square tests of independence  
**Goal**: Identify morphological features that predict mushroom edibility

---

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, chi2
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ Libraries loaded successfully")

In [ ]:
# Load mushroom dataset
df = pd.read_csv('mushroom.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

## 2. Data Understanding and Preprocessing

In [ ]:
# Define feature codes for interpretation
feature_codes = {
    'class': {'e': 'Edible', 'p': 'Poisonous'},
    'cap-color': {
        'n': 'Brown', 'b': 'Buff', 'c': 'Cinnamon', 'g': 'Gray',
        'r': 'Green', 'p': 'Pink', 'u': 'Purple', 'e': 'Red',
        'w': 'White', 'y': 'Yellow'
    },
    'odor': {
        'a': 'Almond', 'l': 'Anise', 'c': 'Creosote', 'y': 'Fishy',
        'f': 'Foul', 'm': 'Musty', 'n': 'None', 'p': 'Pungent', 's': 'Spicy'
    }
}

# Create human-readable versions
df['class_label'] = df['class'].map(feature_codes['class'])
df['cap_color_label'] = df['cap-color'].map(feature_codes['cap-color'])
df['odor_label'] = df['odor'].map(feature_codes['odor'])

print("="*70)
print("DATASET OVERVIEW")
print("="*70)
print(f"\nTotal mushroom samples: {len(df):,}")
print(f"\nClass distribution:")
class_counts = df['class_label'].value_counts()
for class_name, count in class_counts.items():
    pct = (count / len(df)) * 100
    print(f"  {class_name}: {count:,} ({pct:.1f}%)")

print(f"\nNumber of unique cap colors: {df['cap-color'].nunique()}")
print(f"Number of unique odors: {df['odor'].nunique()}")

print(f"\nMissing values:")
print(df[['class', 'cap-color', 'odor']].isnull().sum())

In [ ]:
# Explore cap color distribution
print("\n" + "="*70)
print("CAP COLOR DISTRIBUTION")
print("="*70)
cap_color_dist = df['cap_color_label'].value_counts().sort_values(ascending=False)
for color, count in cap_color_dist.items():
    pct = (count / len(df)) * 100
    print(f"  {color:15s}: {count:5,} ({pct:5.1f}%)")

# Explore odor distribution
print("\n" + "="*70)
print("ODOR DISTRIBUTION")
print("="*70)
odor_dist = df['odor_label'].value_counts().sort_values(ascending=False)
for odor, count in odor_dist.items():
    pct = (count / len(df)) * 100
    print(f"  {odor:15s}: {count:5,} ({pct:5.1f}%)")

## 3. Exploratory Visualization

In [ ]:
# Create exploratory visualizations
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Mushroom Dataset: Exploratory Analysis', fontsize=16, fontweight='bold')

# 1. Cap color by class
pd.crosstab(df['cap_color_label'], df['class_label']).plot(
    kind='bar', ax=axes[0, 0], color=['seagreen', 'indianred'],
    edgecolor='black', linewidth=1.2
)
axes[0, 0].set_title('Cap Color Distribution by Edibility', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Cap Color', fontsize=11)
axes[0, 0].set_ylabel('Count', fontsize=11)
axes[0, 0].legend(title='Class')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Odor by class
pd.crosstab(df['odor_label'], df['class_label']).plot(
    kind='bar', ax=axes[0, 1], color=['seagreen', 'indianred'],
    edgecolor='black', linewidth=1.2
)
axes[0, 1].set_title('Odor Distribution by Edibility', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Odor Type', fontsize=11)
axes[0, 1].set_ylabel('Count', fontsize=11)
axes[0, 1].legend(title='Class')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Proportional stacked bar - cap color
cap_color_pct = pd.crosstab(df['cap_color_label'], df['class_label'], normalize='index') * 100
cap_color_pct.plot(kind='barh', stacked=True, ax=axes[1, 0],
                  color=['seagreen', 'indianred'],
                  edgecolor='black', linewidth=1.2)
axes[1, 0].set_title('Percentage Edible vs Poisonous by Cap Color', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Percentage', fontsize=11)
axes[1, 0].set_ylabel('Cap Color', fontsize=11)
axes[1, 0].legend(title='Class', loc='lower right')
axes[1, 0].grid(axis='x', alpha=0.3)

# 4. Proportional stacked bar - odor
odor_pct = pd.crosstab(df['odor_label'], df['class_label'], normalize='index') * 100
odor_pct.plot(kind='barh', stacked=True, ax=axes[1, 1],
             color=['seagreen', 'indianred'],
             edgecolor='black', linewidth=1.2)
axes[1, 1].set_title('Percentage Edible vs Poisonous by Odor', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Percentage', fontsize=11)
axes[1, 1].set_ylabel('Odor Type', fontsize=11)
axes[1, 1].legend(title='Class', loc='lower right')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('mushroom_exploration.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Exploratory visualizations created")

## 4. Analysis 1: Cap Color vs Edibility

### Hypotheses:
- **H₀**: Cap color and mushroom edibility are independent
- **H₁**: Cap color and mushroom edibility are associated

In [ ]:
# Create contingency table for cap color
cap_color_table = pd.crosstab(df['cap_color_label'], df['class_label'],
                               margins=False)

print("="*70)
print("CONTINGENCY TABLE: Cap Color vs Edibility")
print("="*70)
print(cap_color_table)
print("\n" + "="*70)

# Add totals for reference
cap_color_with_totals = cap_color_table.copy()
cap_color_with_totals['Total'] = cap_color_with_totals.sum(axis=1)
cap_color_with_totals.loc['Total'] = cap_color_with_totals.sum()
print("\nWith Row and Column Totals:")
print(cap_color_with_totals)

In [ ]:
# Perform chi-square test for cap color
chi2_cap, p_cap, dof_cap, expected_cap = chi2_contingency(cap_color_table)

# Calculate Cramér's V
n_cap = cap_color_table.sum().sum()
min_dim_cap = min(cap_color_table.shape[0] - 1, cap_color_table.shape[1] - 1)
cramers_v_cap = np.sqrt(chi2_cap / (n_cap * min_dim_cap))

# Interpret effect size
def interpret_cramers_v(v, df_min):
    if df_min == 1:
        if v < 0.10: return "Negligible"
        elif v < 0.30: return "Small"
        elif v < 0.50: return "Medium"
        else: return "Large"
    else:
        if v < 0.07: return "Negligible"
        elif v < 0.21: return "Small"
        elif v < 0.35: return "Medium"
        else: return "Large"

effect_cap = interpret_cramers_v(cramers_v_cap, min_dim_cap)

print("\n" + "="*70)
print("CHI-SQUARE TEST: CAP COLOR VS EDIBILITY")
print("="*70)
print(f"\nChi-square statistic (χ²): {chi2_cap:.4f}")
print(f"P-value: {p_cap:.10f}")
print(f"Degrees of freedom: {dof_cap}")
print(f"\nEffect Size (Cramér's V): {cramers_v_cap:.4f}")
print(f"Interpretation: {effect_cap} effect")
print("\n" + "="*70)

alpha = 0.05
if p_cap < alpha:
    print("\n✓ REJECT THE NULL HYPOTHESIS")
    print(f"  → Cap color and edibility ARE statistically associated")
    print(f"     (p = {p_cap:.10f} < {alpha})")
else:
    print("\n✗ FAIL TO REJECT THE NULL HYPOTHESIS")
    print(f"  → No significant association found")
    print(f"     (p = {p_cap:.10f} ≥ {alpha})")
print("\n" + "="*70)

In [ ]:
# Expected frequencies and residuals for cap color
expected_cap_df = pd.DataFrame(expected_cap,
                               index=cap_color_table.index,
                               columns=cap_color_table.columns)

print("Expected Frequencies (Cap Color):")
print(expected_cap_df.round(2))

# Check assumptions
min_expected_cap = expected_cap.min()
print(f"\nMinimum expected frequency: {min_expected_cap:.2f}")
if min_expected_cap >= 5:
    print("✓ Assumption satisfied: All expected frequencies ≥ 5")
else:
    print("⚠ WARNING: Some expected frequencies < 5")

# Standardized residuals
residuals_cap = (cap_color_table.values - expected_cap) / np.sqrt(expected_cap)
residuals_cap_df = pd.DataFrame(residuals_cap,
                                index=cap_color_table.index,
                                columns=cap_color_table.columns)

print("\nStandardized Residuals (Cap Color):")
print(residuals_cap_df.round(3))

print("\nCap colors with significant contributions (|z| > 2):")
for i, color in enumerate(cap_color_table.index):
    for j, class_label in enumerate(cap_color_table.columns):
        z = residuals_cap[i, j]
        if abs(z) > 2:
            direction = "MORE" if z > 0 else "FEWER"
            print(f"  • {color} - {class_label}: z = {z:.3f}")
            print(f"    → {direction} than expected")

## 5. Analysis 2: Odor vs Edibility

### Hypotheses:
- **H₀**: Odor and mushroom edibility are independent
- **H₁**: Odor and mushroom edibility are associated

In [ ]:
# Create contingency table for odor
odor_table = pd.crosstab(df['odor_label'], df['class_label'],
                        margins=False)

print("="*70)
print("CONTINGENCY TABLE: Odor vs Edibility")
print("="*70)
print(odor_table)
print("\n" + "="*70)

# Add totals
odor_with_totals = odor_table.copy()
odor_with_totals['Total'] = odor_with_totals.sum(axis=1)
odor_with_totals.loc['Total'] = odor_with_totals.sum()
print("\nWith Row and Column Totals:")
print(odor_with_totals)

In [ ]:
# Perform chi-square test for odor
chi2_odor, p_odor, dof_odor, expected_odor = chi2_contingency(odor_table)

# Calculate Cramér's V
n_odor = odor_table.sum().sum()
min_dim_odor = min(odor_table.shape[0] - 1, odor_table.shape[1] - 1)
cramers_v_odor = np.sqrt(chi2_odor / (n_odor * min_dim_odor))

effect_odor = interpret_cramers_v(cramers_v_odor, min_dim_odor)

print("\n" + "="*70)
print("CHI-SQUARE TEST: ODOR VS EDIBILITY")
print("="*70)
print(f"\nChi-square statistic (χ²): {chi2_odor:.4f}")
print(f"P-value: {p_odor:.10f}")
print(f"Degrees of freedom: {dof_odor}")
print(f"\nEffect Size (Cramér's V): {cramers_v_odor:.4f}")
print(f"Interpretation: {effect_odor} effect")
print("\n" + "="*70)

if p_odor < alpha:
    print("\n✓ REJECT THE NULL HYPOTHESIS")
    print(f"  → Odor and edibility ARE statistically associated")
    print(f"     (p = {p_odor:.10f} < {alpha})")
else:
    print("\n✗ FAIL TO REJECT THE NULL HYPOTHESIS")
    print(f"  → No significant association found")
    print(f"     (p = {p_odor:.10f} ≥ {alpha})")
print("\n" + "="*70)

In [ ]:
# Expected frequencies and residuals for odor
expected_odor_df = pd.DataFrame(expected_odor,
                               index=odor_table.index,
                               columns=odor_table.columns)

print("Expected Frequencies (Odor):")
print(expected_odor_df.round(2))

# Check assumptions
min_expected_odor = expected_odor.min()
print(f"\nMinimum expected frequency: {min_expected_odor:.2f}")
if min_expected_odor >= 5:
    print("✓ Assumption satisfied: All expected frequencies ≥ 5")
else:
    print("⚠ WARNING: Some expected frequencies < 5")

# Standardized residuals
residuals_odor = (odor_table.values - expected_odor) / np.sqrt(expected_odor)
residuals_odor_df = pd.DataFrame(residuals_odor,
                                index=odor_table.index,
                                columns=odor_table.columns)

print("\nStandardized Residuals (Odor):")
print(residuals_odor_df.round(3))

print("\nOdors with significant contributions (|z| > 2):")
for i, odor in enumerate(odor_table.index):
    for j, class_label in enumerate(odor_table.columns):
        z = residuals_odor[i, j]
        if abs(z) > 2:
            direction = "MORE" if z > 0 else "FEWER"
            print(f"  • {odor} - {class_label}: z = {z:.3f}")
            print(f"    → {direction} than expected")

## 6. Comprehensive Visualization of Results

In [ ]:
# Create comprehensive results visualization
fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
fig.suptitle('Chi-Square Analysis: Mushroom Features vs Edibility', 
             fontsize=18, fontweight='bold', y=0.98)

# ROW 1: CAP COLOR ANALYSIS
# 1. Observed frequencies - cap color
ax1 = fig.add_subplot(gs[0, 0])
sns.heatmap(cap_color_table, annot=True, fmt='d', cmap='YlOrRd',
           cbar_kws={'label': 'Count'}, ax=ax1,
           linewidths=1, linecolor='black')
ax1.set_title('Cap Color: Observed Frequencies', fontsize=11, fontweight='bold')
ax1.set_xlabel('Edibility', fontsize=10)
ax1.set_ylabel('Cap Color', fontsize=10)

# 2. Expected frequencies - cap color
ax2 = fig.add_subplot(gs[0, 1])
sns.heatmap(expected_cap_df, annot=True, fmt='.1f', cmap='YlGnBu',
           cbar_kws={'label': 'Expected'}, ax=ax2,
           linewidths=1, linecolor='black')
ax2.set_title('Cap Color: Expected Frequencies', fontsize=11, fontweight='bold')
ax2.set_xlabel('Edibility', fontsize=10)
ax2.set_ylabel('Cap Color', fontsize=10)

# 3. Residuals - cap color
ax3 = fig.add_subplot(gs[0, 2])
sns.heatmap(residuals_cap_df, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
           cbar_kws={'label': 'Std. Residual'}, ax=ax3,
           linewidths=1, linecolor='black', vmin=-10, vmax=10)
ax3.set_title('Cap Color: Standardized Residuals', fontsize=11, fontweight='bold')
ax3.set_xlabel('Edibility', fontsize=10)
ax3.set_ylabel('Cap Color', fontsize=10)

# ROW 2: ODOR ANALYSIS
# 4. Observed frequencies - odor
ax4 = fig.add_subplot(gs[1, 0])
sns.heatmap(odor_table, annot=True, fmt='d', cmap='YlOrRd',
           cbar_kws={'label': 'Count'}, ax=ax4,
           linewidths=1, linecolor='black')
ax4.set_title('Odor: Observed Frequencies', fontsize=11, fontweight='bold')
ax4.set_xlabel('Edibility', fontsize=10)
ax4.set_ylabel('Odor Type', fontsize=10)

# 5. Expected frequencies - odor
ax5 = fig.add_subplot(gs[1, 1])
sns.heatmap(expected_odor_df, annot=True, fmt='.1f', cmap='YlGnBu',
           cbar_kws={'label': 'Expected'}, ax=ax5,
           linewidths=1, linecolor='black')
ax5.set_title('Odor: Expected Frequencies', fontsize=11, fontweight='bold')
ax5.set_xlabel('Edibility', fontsize=10)
ax5.set_ylabel('Odor Type', fontsize=10)

# 6. Residuals - odor
ax6 = fig.add_subplot(gs[1, 2])
sns.heatmap(residuals_odor_df, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
           cbar_kws={'label': 'Std. Residual'}, ax=ax6,
           linewidths=1, linecolor='black', vmin=-30, vmax=30)
ax6.set_title('Odor: Standardized Residuals', fontsize=11, fontweight='bold')
ax6.set_xlabel('Edibility', fontsize=10)
ax6.set_ylabel('Odor Type', fontsize=10)

# ROW 3: SUMMARY STATISTICS
# 7. Cap color summary
ax7 = fig.add_subplot(gs[2, 0:2])
ax7.axis('off')
summary_cap = f"""
CAP COLOR ANALYSIS
{'='*45}

Chi-square: χ² = {chi2_cap:.2f}
P-value: {p_cap:.6f}
Degrees of freedom: {dof_cap}
Cramér's V: {cramers_v_cap:.4f} ({effect_cap})

{'='*45}
Conclusion:
{"✓ Significant association found" if p_cap < 0.05 else "✗ No significant association"}
{"Cap color is associated with edibility" if p_cap < 0.05 else "Cap color and edibility are independent"}
"""
ax7.text(0.1, 0.5, summary_cap, fontsize=10, family='monospace',
        verticalalignment='center',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

# 8. Odor summary
ax8 = fig.add_subplot(gs[2, 2])
ax8.axis('off')
summary_odor = f"""
ODOR ANALYSIS
{'='*35}

Chi-square: χ² = {chi2_odor:.2f}
P-value: {p_odor:.10f}
Degrees of freedom: {dof_odor}
Cramér's V: {cramers_v_odor:.4f}
Effect: {effect_odor}

{'='*35}
Conclusion:
{"✓ STRONG association" if p_odor < 0.05 else "✗ No association"}
{"Odor is a powerful" if p_odor < 0.05 else "Odor is not a"}
{"predictor of toxicity" if p_odor < 0.05 else "reliable predictor"}
"""
ax8.text(0.1, 0.5, summary_odor, fontsize=10, family='monospace',
        verticalalignment='center',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.3))

plt.savefig('mushroom_chi_square_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Comprehensive chi-square analysis visualizations created")

## 7. Comparative Analysis and Interpretation

In [ ]:
# Compare the two features
comparison_data = {
    'Feature': ['Cap Color', 'Odor'],
    'Chi-square': [chi2_cap, chi2_odor],
    'P-value': [p_cap, p_odor],
    'Degrees of Freedom': [dof_cap, dof_odor],
    'Cramers V': [cramers_v_cap, cramers_v_odor],
    'Effect Size': [effect_cap, effect_odor],
    'Significant': [p_cap < 0.05, p_odor < 0.05]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*70)
print("COMPARATIVE ANALYSIS: CAP COLOR VS ODOR")
print("="*70)
print(comparison_df.to_string(index=False))
print("\n" + "="*70)

print("\nKEY INSIGHTS:")
print("\n1. PREDICTIVE POWER:")
if cramers_v_odor > cramers_v_cap:
    ratio = cramers_v_odor / cramers_v_cap
    print(f"   Odor is {ratio:.1f}x stronger predictor than cap color")
    print(f"   (Cramér's V: {cramers_v_odor:.4f} vs {cramers_v_cap:.4f})")
else:
    ratio = cramers_v_cap / cramers_v_odor
    print(f"   Cap color is {ratio:.1f}x stronger predictor than odor")
    print(f"   (Cramér's V: {cramers_v_cap:.4f} vs {cramers_v_odor:.4f})")

print("\n2. STATISTICAL SIGNIFICANCE:")
if p_cap < 0.05:
    print(f"   ✓ Cap color: Significant (p = {p_cap:.6f})")
else:
    print(f"   ✗ Cap color: Not significant (p = {p_cap:.6f})")
    
if p_odor < 0.05:
    print(f"   ✓ Odor: Significant (p = {p_odor:.10f})")
else:
    print(f"   ✗ Odor: Not significant (p = {p_odor:.10f})")

print("\n3. PRACTICAL IMPLICATIONS FOR MUSHROOM IDENTIFICATION:")
if cramers_v_odor > 0.5:
    print("   • Odor is an EXCELLENT indicator of mushroom toxicity")
    print("   • Should be a primary feature in identification")
elif cramers_v_odor > 0.3:
    print("   • Odor is a GOOD indicator of mushroom toxicity")
    print("   • Useful but should be combined with other features")
    
if cramers_v_cap > 0.3:
    print("   • Cap color provides meaningful information")
elif cramers_v_cap > 0.1:
    print("   • Cap color has some predictive value")
else:
    print("   • Cap color alone is not reliable for identification")

print("\n" + "="*70)

## 8. Feature Ranking by Predictive Power

In [ ]:
# Identify most predictive categories within each feature
print("\n" + "="*70)
print("MOST INDICATIVE CHARACTERISTICS")
print("="*70)

print("\nCAP COLORS STRONGLY ASSOCIATED WITH POISONOUS MUSHROOMS:")
for i, color in enumerate(cap_color_table.index):
    z_poisonous = residuals_cap[i, 1]  # Poisonous column
    if z_poisonous > 2:
        print(f"  • {color}: z = {z_poisonous:.2f} (strong positive association)")

print("\nCAP COLORS STRONGLY ASSOCIATED WITH EDIBLE MUSHROOMS:")
for i, color in enumerate(cap_color_table.index):
    z_edible = residuals_cap[i, 0]  # Edible column
    if z_edible > 2:
        print(f"  • {color}: z = {z_edible:.2f} (strong positive association)")

print("\nODORS STRONGLY ASSOCIATED WITH POISONOUS MUSHROOMS:")
for i, odor in enumerate(odor_table.index):
    z_poisonous = residuals_odor[i, 1]  # Poisonous column
    if z_poisonous > 2:
        pct_poisonous = (odor_table.iloc[i, 1] / odor_table.iloc[i].sum()) * 100
        print(f"  • {odor}: z = {z_poisonous:.2f} ({pct_poisonous:.1f}% poisonous)")

print("\nODORS STRONGLY ASSOCIATED WITH EDIBLE MUSHROOMS:")
for i, odor in enumerate(odor_table.index):
    z_edible = residuals_odor[i, 0]  # Edible column
    if z_edible > 2:
        pct_edible = (odor_table.iloc[i, 0] / odor_table.iloc[i].sum()) * 100
        print(f"  • {odor}: z = {z_edible:.2f} ({pct_edible:.1f}% edible)")

print("\n" + "="*70)

## 9. Final Summary and Conclusions

In [ ]:
print("\n" + "#"*70)
print("# FINAL SUMMARY: MUSHROOM CHI-SQUARE ANALYSIS")
print("#"*70)

print("\n1. RESEARCH QUESTIONS:")
print("   a) Is cap color associated with mushroom edibility?")
print("   b) Is odor associated with mushroom edibility?")

print("\n2. DATASET:")
print(f"   • Total samples: {len(df):,}")
print(f"   • Edible: {(df['class']=='e').sum():,} ({(df['class']=='e').mean():.1%})")
print(f"   • Poisonous: {(df['class']=='p').sum():,} ({(df['class']=='p').mean():.1%})")

print("\n3. CAP COLOR FINDINGS:")
print(f"   • Chi-square: χ² = {chi2_cap:.2f}, p = {p_cap:.6f}")
print(f"   • Effect size: Cramér's V = {cramers_v_cap:.4f} ({effect_cap})")
if p_cap < 0.05:
    print("   • ✓ Statistically significant association found")
else:
    print("   • ✗ No significant association")

print("\n4. ODOR FINDINGS:")
print(f"   • Chi-square: χ² = {chi2_odor:.2f}, p < 0.001")
print(f"   • Effect size: Cramér's V = {cramers_v_odor:.4f} ({effect_odor})")
if p_odor < 0.05:
    print("   • ✓ HIGHLY significant association found")
    print("   • Odor is a STRONG predictor of mushroom toxicity")
else:
    print("   • ✗ No significant association")

print("\n5. COMPARATIVE CONCLUSION:")
if cramers_v_odor > cramers_v_cap:
    print(f"   • Odor is a MUCH STRONGER predictor than cap color")
    print(f"   • Effect size ratio: {cramers_v_odor/cramers_v_cap:.1f}:1")
else:
    print(f"   • Both features show similar predictive power")

print("\n6. PRACTICAL RECOMMENDATIONS:")
print("   • NEVER rely on a single feature for mushroom identification")
if cramers_v_odor > 0.5:
    print("   • Odor should be a PRIMARY consideration")
    print("   • Foul, fishy, or pungent odors are strong warning signs")
if cramers_v_cap > 0.2:
    print("   • Cap color provides supplementary information")
print("   • Always consult multiple features and expert guides")
print("   • When in doubt, do NOT consume!")

print("\n7. STATISTICAL METHODS VALIDATION:")
print("   • All expected frequencies met chi-square assumptions (≥5)")
print("   • Large sample size ensures robust results")
print("   • Effect sizes provide practical significance context")

print("\n" + "#"*70)
print("# ANALYSIS COMPLETE")
print("#"*70)
print("\n⚠️  WARNING: This analysis is for educational purposes only.")
print("    NEVER use statistical analysis alone to determine mushroom safety!")
print("    Always consult expert mycologists and field guides.")

---

## Key Takeaways

1. **Odor is a powerful predictor** of mushroom toxicity (very large effect size)
2. **Cap color shows association** with edibility but with smaller effect size
3. **Chi-square tests confirmed** statistically significant relationships
4. **Standardized residuals identified** specific categories driving associations
5. **Multiple features should be used** together for reliable identification
6. **Statistical analysis complements** but never replaces expert knowledge

---